In [ ]:
%matplotlib inline
import pandas as pd
from rich import print
import numpy as np
import matplotlib.pyplot as plt
#from quickbin import bin2d
from scipy.stats import binned_statistic_2d
import pdr
from astropy.visualization import ZScaleInterval
from astropy import wcs as pywcs
import warnings
from pyarrow import ArrowInvalid
from tqdm import tqdm

In [ ]:
import pyarrow.parquet as pq
table = pq.read_table('data/e23456/e23456-nd-f0120-b00-movie-photom-9_0-base.parquet').to_pandas()
data = pdr.read('data/e23456/e23456-nd-f0120-b00-movie-r.fits')
data.load('CNT')

In [ ]:
nbins = data['CNT_HEADER']['N_FRAME']
expt = np.array([data['CNT_HEADER'][f'EXPT_{n}'] for n in range(nbins)])
t = np.arange(nbins)

# plt.figure()
outlier = []
for row in tqdm(table.iterrows()):
    aper_cnts = np.array([row[1][f'aperture_sum_{i}'] for i in range(nbins)])
    flags = np.array([row[1][f'artifact_flag_{i}'] for i in range(nbins)])
    ix = np.where(flags==0)
    if len(ix[0])<=2:
        continue
    cps = aper_cnts/expt
    cps_err = np.sqrt(aper_cnts)/expt
    #plt.plot(t[ix],cps[ix],'k-',alpha=0.1)

    sigma = 3
    second_min = np.sort((cps + cps_err * sigma)[ix])[1]
    second_max = np.sort((cps - cps_err * sigma)[ix])[-2]
    
    # Find outliers above the second minimum (existing logic)
    outlier_ix_high = np.where((cps - cps_err * sigma)[ix] > second_min)[0]
    
    # Find outliers below the second maximum (new logic)
    outlier_ix_low = np.where((cps + cps_err * sigma)[ix] < second_max)[0]
    
    # Combine both types of outliers
    total_outliers = len(outlier_ix_high) + len(outlier_ix_low)
    
    if total_outliers >= 2:
        outlier+=[[total_outliers,row[0]]]
        continue  # skip if there are not 3 significant outliers using the dumbest heuristic


print(len(outlier))#,outlier)
#plt.semilogy()
# plt.show()

<Figure size 640x480 with 0 Axes>

In [ ]:
plt.errorbar(t,cps,yerr=3*cps_err,fmt='kx-')

In [ ]:
ix

In [ ]:
# count the number of aperture_sum columns
bins = len([col for col in table.keys() if col.startswith('aperture_sum_')])
print(bins)
for row in table.iterrows():
    lc = [row[1][f'aperture_sum_{i}'] for i in range(bins)]
    flags = [row[1][f'artifact_flag_{i}'] for i in range(bins)]
    cps = np.array(lc)/np.array(expt)
    print(cps)
    print(flags)
    break